# Geometric Approaches: Inverse Perspective Mapping

**Imports**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

**Load a Rectified Image from the KITTI Dataset**

In [ ]:
image = #TODO: Load the Stuttgart Image from the Geometric > Geometric > Folder file

plt.imshow(image)
plt.show())

In [ ]:
h, w, d = image.shape
w, h, d

### **IPM with 4 Points**

**Define 4 Points of Transformation**

In [ ]:
img = np.copy(image)
src_1 = #TODO: Define the top left point
src_2 = #TODO: Define the bottom left point
src_3 = #TODO: Defien the top right point
src_4 = #TODO: Define the bottom right point

cv2.circle(img, src_1, radius=15, color=(0,0,205),thickness=-1)
cv2.circle(img, src_2, radius=15, color=(0,0,205),thickness=-1)
cv2.circle(img, src_3, radius=15, color=(0,0,205),thickness=-1)
cv2.circle(img, src_4, radius=15, color=(0,0,205),thickness=-1)
plt.imshow(img)
plt.show()

In [ ]:
img2 = np.copy(img)

dst_1 = #TODO: Define the top left point
dst_2 = #TODO: Define the bottom left point
dst_3 = #TODO: Defien the top right point
dst_4 = #TODO: Define the bottom right point

cv2.rectangle(img2, dst_1, dst_4, (205,0,0),4)

plt.imshow(img2)
plt.show()

**Apply the Transformation**

In [ ]:
src_points = np.array([src_1, src_2, src_3, src_4],dtype=np.float32)  # Original Region of Interest
dst_points = np.array([dst_1, dst_2, dst_3, dst_4],dtype=np.float32)   # Projected Rectangle

In [ ]:
np.set_printoptions(precision=4, suppress=True)
M = #TODO: Call the perspective transform function from openCV
print(M)

In [ ]:
warped_img = #TODO: Call the warp perspective function from openCV

plt.imshow(warped_img)
plt.show()

**This is done completely manually. A better alternative is to use the camera intrinsic and extrinsic calibration parameters**

## IPM with Parameters
Later in the course, you're going to apply IPM inside a Spatial Transformer Network. That part will be abstracted later on, but we WILL work on it right now.

First, we load a new image

In [ ]:
filename= #TODO: Load a front image from the Geometric>Geometric>Front Dataset (ideally with no obstacles in front, like t_0_0_0001000.png)

image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

**Next, we load the camera parameters**.
To do this, we'll need:
* The intrinsic calibation matrix K
* The position of the camera
* The Yaw, Pitch, and Roll

In [ ]:
import yaml
with open("/kaggle/input/front-test/geometric/geometric/front/front.yaml") as stream:
    camConfig = yaml.safe_load(stream)
    
print(camConfig)

In [ ]:
# camera calibration matrix K
fx= #TODO: Pick from the config file
fy= #TODO: Pick from the config file
px= #TODO: Pick from the config file
py= #TODO: Pick from the config file

# rotation matrix R (in deg)
yaw= #TODO: Pick from the config file
pitch= #TODO: Pick from the config file
roll= #TODO: Pick from the config file

# vehicle coords of camera origin
XCam= #TODO: Pick from the config file
YCam=#TODO: Pick from the config file
ZCam= #TODO: Pick from the config file

In [ ]:
K = np.zeros([3, 3])
R = np.zeros([3, 3])
t = np.zeros([3, 1])
P = np.zeros([3, 4])

In [ ]:
## Set K

K #TODO: Fill K with matrices from the config file


In [ ]:
y = np.deg2rad(yaw) 
p = np.deg2rad(pitch)
r = np.deg2rad(roll)

## Set R
Rz = #TODO: Implement the matrix for Rz
Ry = #TODO: Implement the matrix for Ry
Rx = #TODO: Implement the matrix for Rx

#Rs = np.array([[0.0, -1.0, 0.0], [0.0, 0.0, -1.0], [1.0, 0.0, 0.0]]) # switch axes (x = -y, y = -z, z = x)
Rs = np.array([[0.0, 1.0, 0.0], [0.0, 0.0, -1.0], [1.0, 0.0, 0.0]]) # switch axes (x = -y, y = -z, z = x) #Additional R matrix

R = #TODO: R = Rs*(Rz*(Ry*Rx))

In [ ]:
## Set T
X = #Define X, an array with X, Y, Z cam
t = #TODO: -R*X

In [ ]:
## Update P
Rt = np.zeros([3, 4])
Rt[0:3, 0:3] = R
Rt[0:3, 3] = t
P = #TODO: Define P

In [ ]:
print(P)

**Transform**

In [ ]:
import sys
sys.path.append("/kaggle/input/front-test/geometric/geometric/")
from utils import Plane, meshgrid, bilinear_sampler, perspective

TARGET_W, TARGET_H = #TODO: Define the size of a plane
plane = Plane(0, -25, 0, 0, 0, 0, TARGET_H, TARGET_W, 0.1)

pixel_coords = #TODO: Call the perspective function
img_bev = #TODO: Upsample the points
plt.imshow(img_bev.astype(int))
plt.show()

#### Try on other images? 
The other folders have their own camera parameters, and images.

In [ ]:
import yaml
import os

filename= '/kaggle/input/front-test/geometric/geometric/rear/rear.png'
image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

with open("/kaggle/input/front-test/geometric/geometric/rear/rear.yaml") as stream:
    camConfig = yaml.safe_load(stream)
    
print(camConfig)

In [ ]:
def get_parameters(camConfig):
    K = np.zeros([3, 3])
    R = np.zeros([3, 3])
    t = np.zeros([3, 1])
    P = np.zeros([3, 4])
    K[0, 0] = camConfig['fx']
    K[1, 1] = camConfig['fy']
    K[0, 2] = camConfig['px']
    K[1, 2] = camConfig['py']
    K[2, 2] = 1.0
    y = np.deg2rad(camConfig['yaw']) 
    p = np.deg2rad(camConfig['pitch'])
    r = np.deg2rad(camConfig['roll'])

    ## Set R
    Rz = np.array([[np.cos(-y), -np.sin(-y), 0.0], [np.sin(-y), np.cos(-y), 0.0], [0.0, 0.0, 1.0]])
    Ry = np.array([[np.cos(-p), 0.0, np.sin(-p)], [0.0, 1.0, 0.0], [-np.sin(-p), 0.0, np.cos(-p)]])
    Rx = np.array([[1.0, 0.0, 0.0], [0.0, np.cos(-r), -np.sin(-r)], [0.0, np.sin(-r), np.cos(-r)]])
    #Rs = np.array([[0.0, -1.0, 0.0], [0.0, 0.0, -1.0], [1.0, 0.0, 0.0]]) # switch axes (x = -y, y = -z, z = x)
    Rs = np.array([[0.0, 1.0, 0.0], [0.0, 0.0, -1.0], [1.0, 0.0, 0.0]]) # switch axes (x = -y, y = -z, z = x)
    R = Rs.dot(Rz.dot(Ry.dot(Rx)))
        
    ## Set T
    X = np.array([camConfig['XCam'], camConfig['YCam'], camConfig['ZCam']])
    t = -R.dot(X)
    
    Rt = np.zeros([3, 4])
    Rt[0:3, 0:3] = R
    Rt[0:3, 3] = t
    P = K.dot(Rt)
    return P

P = get_parameters(camConfig)

**Repeat the Projection Code**

In [ ]:
TARGET_W, TARGET_H = 500, 500
plane = Plane(-50, -25, -25, 0, 0, 0, TARGET_H, TARGET_W, 0.1)

pixel_coords = perspective(plane.xyz, P, TARGET_H, TARGET_W)
img_bev = bilinear_sampler(np.copy(image), pixel_coords)
plt.imshow(img_bev.astype(int))
plt.show()

# **Inverse Projections with Depth**

### Original Code

In [ ]:
import imageio
imageio.plugins.freeimage.download()

depth = #TODO: Load the Depth.EXR image from Depth>Depth
image = #TODO: Load the RGB Image from Depth > Depth

plt.imshow(depth, cmap="inferno")
plt.show()

In [ ]:
print(depth)

In [ ]:
print(image.shape)
print(depth.shape)
height, width = depth.shape[:2]

In [ ]:
def intrinsic_from_fov(height, width, fov=90):
    """
    Basic Pinhole Camera Model
    intrinsic params from fov and sensor width and height in pixels
    Returns:
        K:      [4, 4]
    """     
    px, py = (width / 2, height / 2)
    hfov = fov / 360. * 2. * np.pi
    fx = width / (2. * np.tan(hfov / 2.))

    vfov = 2. * np.arctan(np.tan(hfov / 2) * height / width)
    fy = height / (2. * np.tan(vfov / 2.))

    return np.array([[fx, 0, px, 0.],
                     [0, fy, py, 0.],
                     [0, 0, 1., 0.],
                     [0., 0., 0., 1.]])

intrinsic = intrinsic_from_fov(height, width)

In [ ]:
cam_coords = np.zeros((height * width, 3))

u0 = intrinsic[0, 2]
v0 = intrinsic[1, 2]
fx = intrinsic[0, 0]
fy = intrinsic[1, 1]
i = 0

# Loop through each pixel in the image
for v in range(height):
    for u in range(width):
        x = # TODO: Define X
        y = # TODO: Define Y
        z = # TODO: Define Z
        cam_coords[i] = (x, y, z)
        i += 1
cam_coords = cam_coords.T


In [ ]:
def project_depthmap(cam_points,rgb, cam_pos = -1.2):
    max_longitudinal = #TODO: Define a Max Distance
    window_x = #TODO: Define a Side Window
    window_y = #TODO: Define a Range Window

    x, y, z = cam_points
    # flip the y-axis to positive upwards
    y = - y

    # We sample points for points less than 70m ahead and above ground
    # Camera is mounted 1m above on an ego vehicle
    ind = np.where((z < max_longitudinal) & (y > cam_pos))
    #ind = np.where(z < max_longitudinal)
    bird_eye = cam_points[:3, ind]

    # Color by pixels or radial distance
    dists = np.sqrt(np.sum(bird_eye[0:2:2, :] ** 2, axis=0))
    axes_limit = 10
    colors = np.minimum(1, dists / axes_limit / np.sqrt(2))
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(24, 12))
    ax0.imshow(rgb)
    ax0.set_title("Original Image")
    ax0.axis("off")
    ax1.scatter(bird_eye[0, :], bird_eye[2, :], c=colors, s=0.1)
    ax1.set_xlim(window_x)
    ax1.set_ylim(window_y)
    ax1.set_title('Bird Eye View')
    plt.axis('off')

    plt.gca().set_aspect('equal')
    plt.show()

project_depthmap(cam_coords, image)

### KITTI Challenge

In [ ]:
image = cv2.cvtColor(cv2.imread('/kaggle/input/front-test/depth/depth/KITTI/000009.png'), cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
depth = np.load("/kaggle/input/front-test/depth/depth/KITTI/depth_map_monodepth.npy")
print(depth.shape)
plt.imshow(depth, cmap="magma")

In [ ]:
print(image.shape)
print(depth.shape)
print((depth))

In [ ]:
def disp_to_depth(disp, min_depth, max_depth): 
    '''
    Convert network's sigmoid output into depth prediction 
    The formula for this conversion is given in the 'additional considerations' 
    section of the paper. 
    ''' 
    min_disp = 1 / max_depth 
    max_disp = 1 / min_depth 
    scaled_disp = min_disp + (max_disp - min_disp) * disp 
    depth = 1 / scaled_disp 
    return scaled_disp, depth

scaled_disp, depth = disp_to_depth(depth, 1e-3, 80)

In [ ]:
print(np.max(scaled_disp))

**Get the 3D values**

In [ ]:
disp_resized = cv2.resize(depth, (1216, 352))
depth = 5.4 / disp_resized
depth = np.clip(depth, 0, 80)
#depth = np.uint16(depth * 256)

In [ ]:
print(np.max(depth))

In [ ]:
height, width = depth.shape
intrinsic = intrinsic_from_fov(height, width)
print(intrinsic)

In [ ]:
#Real KITTI Values
#K = [[721.5377   0.     609.5593]
 #[  0.     721.5377 172.854 ]
 #[  0.       0.       1.    ]]

In [ ]:
cam_coords = np.zeros((height * width, 3))

u0 = intrinsic[0, 2] 
v0 = intrinsic[1, 2] 
fx = intrinsic[0, 0] 
fy = intrinsic[1, 1]
i = 0

# Loop through each pixel in the image
for v in range(height):
    for u in range(width):
        x = (u - u0) * depth[v, u] / fx
        y = (v - v0) * depth[v, u] / fy
        z = depth[v, u]
        cam_coords[i] = (x, y, z)
        i += 1
cam_coords = cam_coords.T

In [ ]:
project_depthmap(cam_coords, image, cam_pos=-2)

In [ ]:
def project_topview(cam_points, rgb):
    """
    Draw the topview projection
    """
    max_longitudinal = 70
    window_x = (-50, 50)
    window_y = (-3, max_longitudinal)

    x, y, z = cam_points
    # flip the y-axis to positive upwards
    y = - y

    # We sample points for points less than 70m ahead and above ground
    # Camera is mounted 1m above on an ego vehicle
    ind = np.where((z < max_longitudinal) & (y > -1.2) & (y<2))
    #ind = np.where(z < max_longitudinal)

    bird_eye = cam_points[:3, ind]
    colors_rgb = rgb[cam_points[1].astype(int), cam_points[0].astype(int), :]
    filtered_colors = colors_rgb[ind[0]]/255.0

    # Color by pixels or radial distance
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(24, 12))
    ax0.imshow(rgb)
    ax0.set_title("Original Image")
    ax0.axis("off")
    ax1.scatter(bird_eye[0, :], bird_eye[2, :], c=filtered_colors, s=0.1)
    ax1.set_xlim(window_x)
    ax1.set_ylim(window_y)
    ax1.set_title('Bird Eye View')
    plt.axis('off')

    plt.gca().set_aspect('equal')
    plt.show()


# Do top view projection
project_topview(cam_coords, image)